# Pedestrian example of subscribing to a DataSet

It is possible to *subscribe* to a dataset. Subscribing means adding a function to the dataset and having the dataset call that function every time a result is added to the dataset (or more rarely, see below).

### Call signature

The subscribing function must have the following call signature:
```
fun(results: List[Tuple[Value]], length: int,
    state: Union[MutableSequence, MutableMapping]) -> None:
    """
    Args:
        results: A list of tuples where each tuple holds the results inserted into the dataset.
            For two scalar parameters, X and Y, results might look like [(x1, y1), (x2, y2), ...]
        length: The current length of the dataset.
        state: Any mutable sequence/mapping that can be used to hold information from call to call.
            In practice a list or a dict.
    """
```
Below we provide an example function that counts the number of times a voltage has exceeded a certain limit.

### Frequency

Since calling the function **every** time an insertion is made may be too frequent, a `min_wait` and a `min_count` argument may be provided when subscribing. The dataset will then only call the function upon inserting a result
if `min_wait` seconds have elapsed since the last call (or the start of the subscription, in the time before the first call) and `min_count` results have been added to the dataset since the last call (or the start of the subscription). All the results added in the meantime are queued and passed to the function in one go.

### Order

The subscription must be set up **after** all parameters have been added to the dataset.

In [1]:
from qcodes import initialise_database, load_or_create_experiment, \
    new_data_set, ParamSpec
import numpy as np
from time import sleep

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /home/vsts/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


Qcodes Logfile : /home/vsts/.qcodes/logs/201102-15638-qcodes.log


## Example 1: A notification

We imagine scanning a frequency and reading out a noisy voltage. When the voltage has exceeded a threshold 5 times, we want to receive a warning. Let us initialise our database and create an experiment.

In [2]:
initialise_database()
exp = load_or_create_experiment(experiment_name="subscription_tutorial", sample_name="no_sample")

In [3]:
dataSet = new_data_set("test",
                       exp_id=exp.exp_id,
                       specs=[ParamSpec("x", "numeric", unit='Hz'),
                              ParamSpec("y", "numeric", unit='V')])
dataSet.mark_started()

In [4]:
def threshold_notifier(results, length, state):
    if len(state) > 4:
        print(f'At step {length}: The voltage exceeded the limit 5 times! ')
        state.clear()
    for result in results:
        if result[1] > 0.8:
            state.append(result[1])

In [5]:
# now perform the subscription
# since this is important safety info, we want our callback function called
# on EVERY insertion
sub_id = dataSet.subscribe(threshold_notifier, min_wait=0, min_count=1, state=[])

In [6]:
for x in np.linspace(100, 200, 150):
    y = np.random.randn()
    dataSet.add_result({"x": x, "y": y})

2020-11-02 16:34:25,479 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 110 ¦ /home/vsts/work/1/s/qcodes/utils/deprecate.py:58: QCoDeSDeprecationWarning: The function <add_result> is deprecated. Use "add_results" as an alternative.
  issue_deprecation_warning(f'{t} <{n}>', reason, alternative)



At step 21: The voltage exceeded the limit 5 times! 
At step 39: The voltage exceeded the limit 5 times! 
At step 49: The voltage exceeded the limit 5 times! 
At step 60: The voltage exceeded the limit 5 times! 
At step 84: The voltage exceeded the limit 5 times! 
At step 109: The voltage exceeded the limit 5 times! 
At step 134: The voltage exceeded the limit 5 times! 
At step 146: The voltage exceeded the limit 5 times! 


In [7]:
dataSet.unsubscribe_all()

## Example 2: ASCII Plotter

While this example does not represent a data acqusition that one may deal with in a real experiment, it is a fun practice of subscribing to a dataset.

In [8]:
dataSet = new_data_set("test", exp_id=exp.exp_id,
                       specs=[ParamSpec("blip", "numeric", unit='bit'),
                              ParamSpec("blop", "numeric", unit='bit')])
dataSet.mark_started()

In [9]:
def ASCII_plotter_5bit(results, length, state):
    """
    Glorious 5-bit signal plotter
    
    Digitises the range (-1, 1) with 4 bits and plots it
    in stdout. Crashes and burns if given data outside that
    interval.
    """
    for result in results:
        plotline = ['.'] * 32
        yvalue = result[1]
        yvalue += 1
        yvalue /= 2
        yvalue = int(yvalue*31)
        plotline[yvalue] = 'O'
        print(''.join(plotline))
        

In [10]:
sub_id = dataSet.subscribe(ASCII_plotter_5bit, min_wait=0, min_count=3, state=[])

In [11]:
for x in np.linspace(0, 3*np.pi, 100):
    yvalue = 0.9*np.sin(x) + np.random.randn()*0.05
    dataSet.add_result({"blip": x, "blop": yvalue})
    sleep(0.1)

...............O................
................O...............
..................O.............


...................O............
.....................O..........
......................O.........


........................O.......
.........................O......
...........................O....


............................O...
..............................O.
...............................O


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/share/miniconda/envs/qcodes/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/vsts/work/1/s/qcodes/dataset/data_set.py", line 172, in run
    self._loop()
  File "/home/vsts/work/1/s/qcodes/dataset/data_set.py", line 195, in _loop
    self._call_callback_on_queue_data()
  File "/home/vsts/work/1/s/qcodes/dataset/data_set.py", line 186, in _call_callback_on_queue_data
    self.callback(result_list, self._data_set_len, self.state)
  File "<ipython-input-1-348631bac194>", line 15, in ASCII_plotter_5bit
    plotline[yvalue] = 'O'
IndexError: list assignment index out of range

